In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [2]:
from unsloth import FastLanguageModel
import torch

model,tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=2048,  #maximum number of tokens the model can process in a single input sequence
    load_in_4bit=True,
    dtype=None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
 model,
 r = 16, # LoRA rank
 target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
 lora_alpha = 16,
 lora_dropout = 0, # Optimized at 0
 bias = "none", # No additional bias terms
 use_gradient_checkpointing = "unsloth", # Gradient checkpointing to save memory
 random_state = 3407,
 use_rslora = False, # Rank stabilized LoRA, can be enabled for stability
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt
dataset = load_dataset('fhai50032/Symptoms_to_disease_7k', split="train")


README.md:   0%|          | 0.00/106 [00:00<?, ?B/s]

Symptoms_to_disease_7k.json:   0%|          | 0.00/3.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7043 [00:00<?, ? examples/s]

In [5]:
from unsloth.chat_templates import standardize_sharegpt
import pandas as pd

def format_dataset(example):
  return{
      "conversations":[
          {"from":"system","value":example["query"].split("Patient:")[0].strip()},
          {"from":"user", "value":example["query"].split("Patient:")[1].strip()},
          {"from":"assistant", "value":example["response"]}
      ]
  }

dataset = dataset.map(format_dataset)
dataset = standardize_sharegpt(dataset)

Map:   0%|          | 0/7043 [00:00<?, ? examples/s]

Standardizing format:   0%|          | 0/7043 [00:00<?, ? examples/s]

In [6]:

df = pd.DataFrame(dataset)
df.head()

,query,response,conversations
0,System:'You are a highly qualified doctor.and ...,You may have Jaundice,[{'content': 'System:'You are a highly qualifi...
1,System:'You are a highly qualified doctor.and ...,You may have Hyperthyroidism,[{'content': 'System:'You are a highly qualifi...
2,System:'You are a highly qualified doctor.and ...,You may have Dimorphic Hemmorhoids(Piles),[{'content': 'System:'You are a highly qualifi...
3,System:'You are a highly qualified doctor.and ...,You may have Tuberculosis,[{'content': 'System:'You are a highly qualifi...
4,System:'You are a highly qualified doctor.and ...,You may have Alcoholic Hepatitis,[{'content': 'System:'You are a highly qualifi...


In [7]:
def tokenize_dataset(prompt):
  convos = prompt["conversations"]
  texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)  for convo in convos]
  return {"text":texts}

dataset = dataset.map(tokenize_dataset, batched = True)

Map:   0%|          | 0/7043 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
 model = model,
 tokenizer = tokenizer,
 train_dataset = dataset,
 dataset_text_field = "text",
 max_seq_length = 2048,
 data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
 dataset_num_proc = 2,
 packing = False,
 args = TrainingArguments(
 per_device_train_batch_size = 2,
 gradient_accumulation_steps = 4,
 warmup_steps = 5,
 max_steps = 60,
 learning_rate = 2e-4,
 fp16 = not is_bfloat16_supported(),
 bf16 = is_bfloat16_supported(),
 logging_steps = 1,
 optim = "adamw_8bit",
 weight_decay = 0.01,
 lr_scheduler_type = "linear",
 seed = 3407,
 output_dir = "outputs",
 ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/7043 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/7043 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/7043 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/7043 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
 trainer,
 instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
 response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

trainer_stats = trainer.train()
#tells model to focus on these parts for training


Map:   0%|          | 0/7043 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,043 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: antonprince95 (antonprince95-school-of-engineering-cusat) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,4.681600
2,3.990700
3,4.374400
4,3.270500
5,2.297700
6,1.713600
7,1.123600
8,0.965900
9,0.491500
10,0.925900


In [13]:
messages = [
    {"role":"user","content":"'I may have chills,vomiting,fatigue,weight_loss,cough,high_fever,sweating,loss_of_appetite,mild_fever,yellowing_of_eyes,swelled_lymph_nodes,malaise,phlegm,chest_pain,blood_in_sputum"}
]
FastLanguageModel.for_inference(model)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens=1024, use_cache=True, temperature=1.5, min_p=0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 01 Mar 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'I may have chills,vomiting,fatigue,weight_loss,cough,high_fever,sweating,loss_of_appetite,mild_fever,yellowing_of_eyes,swelled_lymph_nodes,malaise,phlegm,chest_pain,blood_in_sputum<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYou may have Tuberculosis<|eot_id|>"]

In [14]:
model.save_pretrained("fine_tuned_llama")
tokenizer.save_pretrained("fine_tuned_llama")

('fine_tuned_llama/tokenizer_config.json',
 'fine_tuned_llama/special_tokens_map.json',
 'fine_tuned_llama/tokenizer.json')